# clj-leaflet

An exploration of ipyleaflet compatible models for clojupyter kernel.

## Setup
To generate the models, this notebook requires a clojupyter kernel with ipywidgets support. Compile and install the latest version [feature/ipywidgets](https://github.com/clojupyter/clojupyter/tree/feature/ipywidgets) branch.

To view and interact with the models you need to install the jupyter lab/notebook extensions for [ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet) and [ipywidgets](https://github.com/jupyter-widgets/ipywidgets).

In [1]:
(require '[clojupyter.widgets.ipywidgets :as ipy])
(require '[clojure.data.json :as json])
(require '[clojure.java.io :as io])
(require '[clojupyter.kernel.comm-atom :as ca])
(require '[clojupyter.state :as st])
(require '[clojupyter.util-actions :as u])
(require '[clojure.repl :refer :all])
(require '[clojure.pprint :refer [pprint]])
(require '[camel-snake-kebab.core :as csk]);

Hack to fix an incompatibility between latest version of jupyterlab manager and jupyter-leaflet.
In order to use an older version of jupyterlab manager we need to enforce the generated widget versions to an earlier version.

In [2]:
(defn old
  ([constr] (old constr {}))
  ([constr state] (constr (merge state {:_model_module_version "1.5.0" :_view_module_version "1.5.0"}))))
#_(defn old
  ([constr] (old constr {}))
  ([constr state] (constr (merge state {}))))

#'user/old

Helper macro useful to attach widgets to global vars and show them in a single command.

In [3]:
(defmacro def-show
  "Like def, but returns `init` if one was passed along."  
  ([symb]             `(def ~symb))
  ([symb init]         `(do (def ~symb ~init) ~symb))
  ([symb docstr init] `(do (def ~symb ~docstr ~init) ~symb)))

#'user/def-show

----
## Leaflet Factory

### Colors
Choropleth models in python use an extenal dependency to compute colors from values.
We define single-hue-linear-fn at the bottom of this section to achieve the same goal.

In [4]:
(defn abs
  [x]
  (if (neg? x) (- x) x))

#'user/abs

In [5]:
(defn str->rgb
  "Converts a string colour to a three int rbg tuple"
  [s]
  (->> (rest s)
       (partition 2)
       (map (partial reduce str))
       (map #(Integer/parseInt % 16))))

#'user/str->rgb

In [6]:
(defn rgb->hsv
  [clr]
  (let [[r g b] (map #(/ % 255) clr)
        c-max (max r g b)
        c-min (min r g b)
        dif (- c-max c-min)
        v c-max
        s (if (zero? dif) 0 (/ dif c-max))
        h (if (zero? dif)
            0
            (condp = c-max
              r (* 60 (mod (/ (- g b) dif) 6))
              g (* 60 (+ 2 (/ (- b r) dif)))
              b (* 60 (+ 4 (/ (- r g) dif)))))]
    (map float (list h s v))))

#'user/rgb->hsv

In [7]:
(defn hsv->rgb
  [[h s v]]
  {:pre [(<= 0 h 360)
         (<= 0 s 1)
         (<= 0 v 1)]}
  (let [c (* v s)
        q (/ h 60)
        x (* c (- 1 (abs (dec (mod q 2)))))
        [r1 g1 b1] (condp #(<= (first %1) %2 (second %1)) q
                     [0 1] [c x 0]
                     [1 2] [x c 0]
                     [2 3] [0 c x]
                     [3 4] [0 x c]
                     [4 5] [x 0 c]
                     [5 6] [c 0 x]
                     :else [0 0 0])
        m (- v c)]
    (map (comp int (partial * 255) (partial + m)) [r1 g1 b1])))

#'user/hsv->rgb

In [8]:
(defn rgb->str
  [[r g b]]
  {:pre [(every? int? [r g b])
         (every? #(<= 0 % 255) [r g b])]}
  (format "#%02X%02X%02X" r g b))

#'user/rgb->str

In [9]:
(defn single-hue-linear-fn
  ([v-min v-max] (single-hue-linear-fn v-min v-max "#BE481F"))
  ([v-min v-max color]
   (fn [x]
     {:pre [(<= v-min x v-max)
            (< v-min v-max)]}
     (let [[h _ v] (-> color str->rgb rgb->hsv)]
       (-> (list h (float (/ (- x v-min) (- v-max v-min))) v)
           hsv->rgb
           rgb->str)))))

#'user/single-hue-linear-fn

#### Color transform test

In [10]:
(let [pc (old ipy/color-picker)
      lab (old ipy/label {:value (:value @pc)})
      _ (.watch pc :on-change-value (fn [_ _ {o-val :value} {n-val :value}]
                                      (when (not= o-val n-val)
                                        (swap! lab assoc :value (-> n-val str->rgb rgb->hsv pr-str)))))]
  (old ipy/v-box {:children [pc lab]}))  

[3a4e71d6-029e-43c8-b07d-cf49bf766f8f]=VBoxModel

### Models

In [11]:
(def SPECS (-> "resources/leaflet-schema.min.json"
               #_io/resource
               slurp
               json/read-str))

#'user/SPECS

In [12]:
(defn def-widget
  [{attributes "attributes"}]
  (let [all-attr (->> attributes (map #(get % "name")) (map keyword) (filterv (partial not= :layers)))]
    (reduce merge
      (for [{name "name" default "default" type "type"} attributes]
        {(keyword name) (cond
                          (= name "options") all-attr
                          (= type "reference") nil
                          :else default)}))))

#'user/def-widget

---
## Model Tests

We import the base maps json file, flatten it and transform its keys to keywords to make it easier to work with.
The original python dict `{"open-street-map" {"mapnick" {"base-map dict"}}}` becomes `{:open-street-map-mapnick {"base-map dict"}}`

In [13]:
(def BASE-MAPS
  (let [base-maps (json/read-str (slurp "./resources/basemaps.json"))
        k-maps (for [[outer-key v] base-maps]
                 (if (contains? v "name")
                   {(csk/->kebab-case-keyword outer-key) (clojure.walk/keywordize-keys v)}
                   (reduce merge
                           (for [[inner-key vv] v]
                             {(csk/->kebab-case-keyword (str outer-key inner-key)) (clojure.walk/keywordize-keys vv)}))))]
    (reduce merge k-maps)))

#'user/BASE-MAPS

**Note:** The base-map hash-map :url must be formatted with a date string to generate the right value.

In [14]:
(declare tile-layer zoom-control attribution-control)

#'user/attribution-control

In [15]:
(defn make-widget
  [spec]
  (fn constructor
    [& args]
    (let [d-state (def-widget spec)
          state (merge d-state (apply hash-map args))
          target "jupyter.widget"
          {jup :jup req-msg :req-message} (st/current-context)
          id (u/uuid)
          v-keys (set (keys d-state))]
      (if (= (get spec "name") "map")
        (let [{:keys [basemap modisdate extra-controls?]
               :or {basemap :open-street-map-mapnik
                    modisdate (:modisdate d-state)
                    extra-controls? true}} args
              state (assoc state :basemap basemap :extra-controls? extra-controls?)]
          (if basemap
            (let [{:keys [url] :as bm} (get BASE-MAPS basemap)
                  date (if (= "yesterday" modisdate)
                         (doto (java.time.LocalDate/now)
                               (.minusDays 1)
                               (.toString))
                         modisdate)
                  bm_ (apply tile-layer (reduce-kv (fn [acc k v]
                                                     (concat acc [k (if (and (= k :url)
                                                                        (clojure.string/includes? v "%s"))
                                                                      (format v date)
                                                                      v)]))
                                               []
                                               bm))
                  state (if extra-controls?
                          (update state :controls (comp vec concat) [(zoom-control) (attribution-control :position "bottomright")])
                          state)]
              (ca/create-and-insert jup req-msg target id v-keys (update state :layers #(into [bm_] %))))
            (ca/create-and-insert jup req-msg target id v-keys (if extra-controls?
                                                                 (update state :controls (comp vec concat) [(zoom-control) (attribution-control :position "bottomright")])
                                                                 state))))  
              ;; At this point we have the basemap widget
              ;; We might want to add by default: zoom-control & attribution control
              ;; We might want to add default map-style widgets
              ;; Are we going to watch the `:modisdate` & `:basemap` attributes to replace the base map layer?
        (ca/create-and-insert jup req-msg target id v-keys state)))))

#'user/make-widget

In [16]:
(for [{n "name" :as spec} SPECS]
  (eval `(def ~(if (= n "map") 'Map (symbol n)) ~(make-widget spec))))

(#'user/ant-path #'user/attribution-control #'user/awesome-icon #'user/choropleth #'user/circle #'user/circle-marker #'user/control #'user/draw-control #'user/feature-group #'user/full-screen-control #'user/geo-data #'user/geo-json #'user/heatmap #'user/icon #'user/image-overlay #'user/layer #'user/layer-group #'user/layers-control #'user/legend-control #'user/local-tile-layer #'user/Map #'user/map-style #'user/marker #'user/marker-cluster #'user/measure-control #'user/path #'user/polygon #'user/polyline #'user/popup #'user/raster-layer #'user/rectangle #'user/scale-control #'user/search-control #'user/split-map-control #'user/tile-layer #'user/ui-layer #'user/vector-layer #'user/vector-tile-layer #'user/video-overlay #'user/widget-control #'user/wms-layer #'user/zoom-control #'user/Layout #'user/DOMWidget)

In [17]:
(defn choropleth
  [& {:keys [_data color-fn key-path data] :or {_data {} key-path [:id] data {}} :as args}]
  (let [base-color "#E96622"
        def-color-fn (fn [data]
                       (if (empty? (vals data))
                         (constantly "#E96622")
                         (let [v-min (reduce min (vals data))
                               v-max (reduce max (vals data))]
                           (if (= v-min v-max)
                             (constantly "#E96622")
                             (single-hue-linear-fn v-min v-max "#E96622")))))
        color-fn (or color-fn def-color-fn)
        color-feature (fn [feat data c-fn key-path]
                        (let [k (get-in feat key-path)]
                          (if-let [v (get data k)]
                            (assoc-in feat [:properties :style] {:fillColor (c-fn v) :color "black" :weight 0.9})
                            feat)))
        color-geo-data (fn [{type :type :as geo-data} data c-fn key-path]
                        (case type
                           "Feature" (color-feature geo-data data c-fn key-path)
                           "FeatureCollection" (update geo-data :features (fn [feat] (mapv #(color-feature % data c-fn key-path) feat)))
                           geo-data))
        data (clojure.walk/keywordize-keys data)
        state (assoc args
                :color-fn color-fn
                :data (color-geo-data data _data (color-fn _data) key-path)
                :_data _data
                :key-path key-path)
        d-state (def-widget (first (filter (comp (partial = "choropleth") #(get % "name")) SPECS)))
        state (merge d-state state)
        w (apply geo-json (reduce concat (list* state)))]
    (.watch w :internal-consistency-data
      (fn [_ r {o-data :data} {:keys [_data color-fn key-path] n-data :data}]
        (when (not= o-data n-data)
          (swap! r update :data color-geo-data _data (color-fn _data) key-path))))
    (.watch w :internal-consistency-_data
      (fn [_ r {o-_data :_data} {:keys [color-fn key-path] n-_data :_data}]
        (when (not= o-_data n-_data)
          (swap! r update :data color-geo-data n-_data (color-fn n-_data) key-path))))
    (.watch w :internal-consistency-key-path
      (fn [_ r {o-key-path :key-path} {:keys [data _data color-fn] n-key-path :key-path}]
        (when (not= o-key-path n-key-path)
          (swap! r update :data color-geo-data _data (color-fn _data) n-key-path))))
    (.watch w :internal-consistency-color-fn
      (fn [_ r {o-color-fn :color-fn} {:keys [_data key-path] n-color-fn :color-fn}]
        (when (not= o-color-fn n-color-fn)
          (swap! r update :data color-geo-data _data (n-color-fn _data) key-path))))
    (ca/insert w)))    

#'user/choropleth

Whatch out for this attribute when building the basemap

### Map Controls

In [18]:
(Map)

[8996acdb-1548-4c28-9905-4c507a418dff]=LeafletMapModel

In [19]:
(def-show M0
  (let [sel (old ipy/dropdown {:options (->> BASE-MAPS keys (map name) sort vec) :description "Base Map"})
        b-maps (reduce merge (for [[k v] BASE-MAPS] {k (apply tile-layer (-> v (update :url format "2020-05-01") list* flatten))}))
        w-ctrl (widget-control :widget sel :position "topright")
        dm (Map :center [4 112] :zoom 3.6 :scroll_wheel_zoom true :basemap :open-topo-map
                :controls [(scale-control :position "bottomleft") (full-screen-control) w-ctrl (search-control)])
        _ (.watch sel :on-select (fn [_ _ {o-val :value} {n-val :value}]
                                   (when (not= o-val n-val)
                                     (swap! dm update :layers (comp #(conj % (get b-maps (keyword n-val))) pop)))))]  
    dm))

[d90e4c38-913e-4000-b9e5-fc46ca4f5f2b]=LeafletMapModel

In [20]:
(def-show M1
  (let [l0 (apply tile-layer (reduce concat (apply list (:esri-world-imagery BASE-MAPS))))
        l1 (apply tile-layer (flatten (apply list (:nasagibs-viirs-earth-at-night-2012 BASE-MAPS))))
        sm (split-map-control :left_layer l0 :right_layer l1)
        m (Map :controls [sm] :center [46, -35] :zoom 3 :basemap nil)]
    m))

[55281c0a-1684-4444-80e1-ad81c5d3b5c8]=LeafletMapModel

In [21]:
(let [dc (draw-control)
      mc (measure-control :primary_length_unit "kilometers" :primary_area_unit "hectares")]
  (Map :center [38 27] :zoom 6 :controls [dc mc] :extra-controls? nil :basemap :carto-db-dark-matter))

[48fcc754-8e7b-49eb-b4c0-e0d3ee5abb39]=LeafletMapModel

**TODO:** search-control example.

### Markers

Awesome-icon widget is not dynamic. You must refer to a new one to change it.

In [22]:
(def-show M2
  (let [loc [52.205 360.121]
        ico (awesome-icon :name "address-card" :title ":boo")
        pin (marker :location loc :draggable true :rise_on_hover true :icon ico)
        lab (old ipy/label {:value (str "Pin at " (:location @pin))})
        m (Map :center loc :zoom 15 :layers [pin])
        text (old ipy/text {:description "Icon Tooltip"})
        _ (.watch text :on-change-value
             (fn [_ _ {o-val :value} {n-val :value}]
               (when (not= o-val n-val)
                 (swap! ico assoc :name n-val))))
        boolin (old ipy/checkbox {:description "Spin?"})
        l0 (old ipy/directional-link {:source [boolin :value] :target [ico :spin]})
        l1 (old ipy/directional-link {:source [text :value] :target [pin :title]})
        _ (.watch pin :on-change-location (fn [_ _ {o-loc :location} {n-loc :location}]
                                           (when (not= o-loc n-loc)
                                             (swap! lab assoc :value (str "Pin at " (reduce conj [] (map (partial format "%.4f") n-loc)))))))
        in-b (old ipy/h-box {:children [text boolin]})]
  (old ipy/v-box {:children [in-b m lab]})))

[96fdc902-a76e-4d85-ba3c-d06de0066cc0]=VBoxModel

### Paths
#### Ant-Path

In [23]:
(def-show M4
  (let [ap [(ant-path :locations [[51.5 8] [52.5 12.1]] :use "polyline")
            (ant-path :locations [[52 8] [52.5 8] [52.3 9]] :use "polygon" :dash-array [50 100] :weight 10 :color "red")
            (ant-path :locations [[51.5 11] [52 12]] :use "rectangle" :dash_array [10 20] :weight 5 :color "white" :pulse_color "green")            
            (ant-path :locations [51.5 10] :radius 15000 :use "circle" :dash_array [5 10] :weight 5 :color "magenta")]]
   (Map :center [52 10] :zoom 8 :layers ap)))

[a0d88fd9-5373-485a-a16f-cc66b2e3d323]=LeafletMapModel

#### Polylines

In [24]:
(let [p0 (polyline :locations [[45.51 -122.68] [37.77 -122.43] [34.04 -118.22]] :color "green" :fill false)
      p1 (polyline :locations [[[45.51 -122.68] [37.77 -122.43] [34.04 -118.22]]
                              [[40.78 -73.91] [41.83 -87.62] [32.76 -96.72]]]
                  :color "green" :fill false)
      p2 (polygon :locations [[42 -60] [48 -34] [32 -48]] :color "green" :fill_color "green")
      p3 (polygon :locations [[[37 -109.05] [41 -109.03] [41 -102.05] [37 -102.04]]
                             [[37.29 -108.58] [40.71 -108.58] [40.71 -102.50] [37.29 -102.50]]]
                 :color "green" :fill_color "green")
      p4 (rectangle :bounds [[52 -6] [53 0]])
      p5 (circle :location [50 -6] :radius 5000 :color "green" :fill_color "green")
      p6 (circle-marker :location [55 0] :radius 50 :color "red" :fill_color "red")
      b-in (old ipy/toggle-buttons {:options ["Polyline" "Multi-Polyline" "Polygon" "Multi-Polygon" "Rectangle" "Circle" "Circle-Marker"]})
      paths [p0 p1 p2 p3 p4 p5 p6]
      switch-lay (fn [lays n-idx] (-> lays pop (conj (get paths n-idx))))
      m (Map :center [42.5 -41] :zoom 3 :layers [p0])
      _ (.watch b-in :on-change-index (fn [_ _ {o-idx :index} {n-idx :index}]
                                        (when (not= o-idx n-idx)
                                          (swap! m update :layers switch-lay n-idx))))]
  (old ipy/v-box {:children [b-in m]}))

[85c4a273-d1bc-4d15-ba8c-9f424dc1f8ac]=VBoxModel

### Audio / Video Overlay

In [25]:
(let [v (video-overlay :url "https://www.mapbox.com/bites/00188/patricia_nasa.webm" :bounds [[13 -130] [32 -100]])
      m (Map :center [23 -115] :zoom 4 :layers [v])]
  m)

[9756a8fd-cba0-4f87-a99e-b487bbd27955]=LeafletMapModel

---
## Geo Data

In [26]:
(def GEO-DATA (json/read-str (slurp "./resources/data/world_bank/WB_countries_Admin0_lowres.geo.json")))
(def POP-MAP (reduce merge (map #(hash-map (get-in % ["properties" "ISO_A3"]) (get-in % ["properties" "POP_EST"])) (get GEO-DATA "features"))));

Geo Data example uses pandas to load a dataset into memory. We need to identify its serial form to replicate on clojure side.

### Geo-JSON

In [27]:
(def-show M3
  (let [lab (old ipy/label)
        h (fn [_ msg _] (swap! lab assoc :value (get-in msg [:properties :WB_NAME] "")))
        l (geo-json :data (clojure.walk/keywordize-keys GEO-DATA) :hover_style {:fillColor "red"} :callbacks {:on-mouseover h}
                     :style {:color "grey" :weight 1 :fillColor "lightyellow" :fillOpacity 0.7})
        m (Map :center [48.4 17.45] :zoom 4 :basemap :open-stree-map-black-and-white :layers [l] :tooltip lab)]
  (old ipy/v-box {:children [m lab]})))

[38b074a1-a76c-4a45-9656-47d7b1be9309]=VBoxModel

### Choropleth
Choropleth model is a geo-json with custom coloring based on some data.
Python model has external dependency for choosing the colours.

In [28]:
(def-show M5
  (let [lab (old ipy/label)
        c (choropleth :data GEO-DATA :_data POP-MAP :key-path [:properties :ISO_A3]
                      :style {:fillOpacity 0.3 :dashArray "5, 5"}
                      :callbacks {:on-mouseover (fn [ref {{name :WB_NAME id :ISO_A3} :properties} _]
                                                  (swap! lab assoc :value (format "%s. Population: %,d" name (get (:_data @ref) id))))})]
    (Map :layers [c] :zoom 2 :center [50 20] :scroll_wheel_zoom true
         :controls [(widget-control :widget lab :position "topright")])))

[0d287fb8-f636-4266-8215-5085c275bcf0]=LeafletMapModel

In [29]:
(def tt0 (System/currentTimeMillis))

#'user/tt0

In [30]:
(let [v-max (reduce max (vals POP-MAP))
      v-min (reduce min (vals POP-MAP))
      col-fn (single-hue-linear-fn v-min v-max "#D61717")
      lab (old ipy/label)
      c (choropleth :data GEO-DATA :_data POP-MAP :style {:fillOpacity 0.6 :dashArray "5, 5"}
                    :key-path [:properties :ISO_A3]
                    :color-fn (fn [m] (single-hue-linear-fn (reduce min (vals m))
                                                            (reduce max (vals m))
                                                            "#D61717"))
                    :callbacks {:on-mouseover (fn [_ {{name :WB_NAME id :ISO_A3} :properties} _]
                                                (swap! lab assoc :value (format "Population %s: %,d" name (get POP-MAP id))))})
      leg-k (take 8 (iterate (partial + (quot (- v-max v-min) 7)) v-min))
      leg (legend-control :position "bottomleft" :legend (zipmap leg-k (map col-fn leg-k)))
      m (Map :layers [c] :center [45 90] :zoom 3 :scroll_wheel_zoom true 
             :controls [(full-screen-control) (widget-control :widget lab :position "topright") leg])]
  m)  

[0eeb1480-18d2-405a-aab2-3086e428ecf0]=LeafletMapModel

### Heatmap

In [31]:
(def tt1 (System/currentTimeMillis))

#'user/tt1

In [32]:
(- tt1 tt0)

2386

In [33]:
(let [loc (vec (repeatedly 1000 #(vector (- (rand 160) 80) (- (rand 360) 180) (rand 1000))))
      hm (heatmap :locations loc :radius 20 :opacity 0.9)
      m (Map :center [0 0] :zoom 2 :extra-controls? false :layers [hm])]
 m)

[3131167d-014f-4d47-b302-75720ec76f3d]=LeafletMapModel

### Velocity
**TODO:**

---
## Conclusions

### Layer Groups / Marker Clusters

Do we need them? Can we use a simple collection of layers/markers?

### Abstract Classes

control, feature-group, layer, layers-control, path, raster-layer, ui-layer, vector-layer, vector-tile-layer, Layout and dom-widgets look like abstract classes. Does it make sense to keep them on clojure side?

----
## Debug

In [34]:
#_(taoensso.timbre/set-level! :debug)

In [35]:
#_(let [p (polyline :locations [[[45.51 -122.68] [37.77 -122.43] [34.04 -118.22]]
                              [[40.78 -73.91] [41.83 -87.62] [32.76 -96.72]]]
                  :color "green" :fill false)
      m (Map :center [42.5 -41] :zoom 2 :options [:center :zoom] :layers [(tile-layer) p])]
  m)

In [36]:
#_(let [p (polygon :locations [[42 -49] [43 -49] [43 -48]] :color "green" :fill_color "green")
      m (Map :center [42.5531 -48.6914] :zoom 6 :options [:center :zoom] :layers [(tile-layer) p])]
  m)

In [37]:
#_(let [p (polygon :locations [[[37 -109.05] [41 -109.03] [41 -102.05] [37 -102.04]]
                             [[37.29 -108.58] [40.71 -108.58] [40.71 -102.50] [37.29 -102.50]]]
                 :color "green" :fill_color "green")
      m (Map :center [37.5531 -109.6914] :zoom 5 :options [:center :zoom] :layers [(tile-layer) p])]
  m)

In [38]:
#_(let [pr (rectangle :bounds [[52 354] [53 360]])
      pc (circle :location [50 354] :radius 50 :color "green" :fill_color "green")
      m (Map :center [53 354] :zoom 5 :options [:center :zoom] :layers [(tile-layer) pr pc])]
  m)

In [39]:
#_(let [cm (circle-marker :location [55 360] :radius 50 :color "red" :fill_color "red")
      m (Map :center [53 354] :zoom 5 :options [:center :zoom] :layers [(tile-layer) cm])]
  m)

In [40]:
#_(->> [(ant-path :locations [[51.5 8] [52.5 12.1]] :use "polyline" :options [:use])
      (ant-path :locations [[52 8] [52.5 8] [52.3 9]] :use "polygon" :dash-array [50 100] :weight 10 :color "red")
      (ant-path :locations [[51.5 11] [52 12]] :use "rectangle" :dash_array [10 20] :weight 5 :color "white" :pulse_color "green")
      (ant-path :locations [51.5 10] :radius 15000 :use "circle" :dash_array [5 10] :weight 5 :color "magenta")]
     last
     (swap! M3 update :layers conj))

In [41]:
#_(clojure.pprint/pprint
 (filter string?
   (for [{n "name" att "attributes"} SPECS
         {a-name "name" a-type "type" nilable? "allow_none"} att]
     (when (= a-type "reference")
       (str "Reference found on widget " n ", attribute " a-name ", nilable? " nilable?)))))